In [1]:
import import_ipynb
import CA
import pprint
CAPubKeyC = []
reqFrame = {}
clientName = 'Tester'
role = 2
ClientCert = []
RecCert = []
clientPubKey = []
clientPriKey = []

importing Jupyter notebook from CA.ipynb


In [2]:
def CertificateRoutine():
    global ClientCert, clientPubKey, clientPriKey, CAPubKeyC, reqFrame, clientName, role
    import socket
    import pickle
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(('127.0.0.7', 8080))
    global ClientCert
    if CAPubKeyC == []:
        reqFrame = {"req":1} # requesting for CA's PublicKey
        data = pickle.dumps(reqFrame)
        print("Requesting for CA's PublicKey \t:\t")
        pprint.pprint(data)
        print('\n')
        client.send(data)
        data = client.recv(4096)
        CAPubKeyC = pickle.loads(data)
        print("CA's PublicKey \t:\t")
        pprint.pprint(data)
        print('\n')
    if ClientCert == []: #Generating Subject'S Public,Private Key pair
        if clientPubKey == []:
            clientPubKey , clientPriKey = CA.KeyGen()
        reqFrame = {"req" : 2, "subName" : clientName, "role":role, "pubKey" : clientPubKey} #Request for certificate
        data = pickle.dumps(reqFrame)
        print("Request for Certificate Generation \t:\t")
        pprint.pprint(data)
        print('\n')
        client.send(data)
        data = client.recv(4096)
        fromServer = pickle.loads(data)
#         print(fromServer)
        ClientCert = CA.certificate()
        ClientCert.dict2cert(fromServer)
        print("Generated Certificate \t:\t")
        pprint.pprint(ClientCert.transmitCertificate())
        print('\n')
    client.close()
CertificateRoutine()

Requesting for CA's PublicKey 	:	
b'\x80\x04\x95\x0c\x00\x00\x00\x00\x00\x00\x00}\x94\x8c\x03req\x94K\x01s.'


CA's PublicKey 	:	
(b'\x80\x04\x95g\x00\x00\x00\x00\x00\x00\x00\x8c\x07rsa.key\x94\x8c\tPublicKey'
 b'\x94\x93\x94)\x81\x94\x8aA\xa7\x8dO\xb6\xdb\xd8\x18\xa8F{m\x9b\x80\x1c\xb4/'
 b'\x0e\xa6\xfe\xd7:\xe3\xb1\xa6*T\xff\xbe\xac8\xfbi0\x188p3\x84\xa7\xe3'
 b'B\x9a2\xaf\x17\xce\xddc"\x01\xfe\x81\x8cF?\xb7\xc4,>b\x0bo8\x9b\x00J\x01\x00'
 b'\x01\x00\x86\x94b.')


Request for Certificate Generation 	:	
(b'\x80\x04\x95\x98\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x03req\x94'
 b'K\x02\x8c\x07subName\x94\x8c\x06Tester\x94\x8c\x04role\x94K\x02\x8c\x06pubK'
 b'ey\x94\x8c\x07rsa.key\x94\x8c\tPublicKey\x94\x93\x94)\x81\x94\x8aA'
 b'?\xe3\x12\x83\x88xdk\xc2\xdfQ\x99\xdeO\\\xc6\x1d\xd5\xe8\xb1\xa9\xbad\xf4'
 b'\x1f\x00\x8fJ\xda\xed\x85X\x7fzhJoq\\\xf1\xcd\xc8s3/\x05\xf0\x10'
 b'a\xe1\xf8\xf8\xa5\xf9`DC\x98\x07\x98\xd4\xc5A\xa9\x00J\x01\x00'
 b'\x01\x00\x86\x94bu.')


Generated Certificate 	:	
{'

In [3]:

import numpy as np
def RandomGenerator():
    random_bytes = str(np.random.bytes(10).hex())
    seed = random_bytes
#     for i in range(0,2):
#         random_bytes = str(np.random.bytes(1).hex())
#         seed = seed +' '+ random_bytes
    return seed.upper()
def Req(data):
    global ClientCert, clientPriKey
    if data["sFn"] == '08':
        if data['Cnf'] == '01':
            print("Authentication using Challenge Response Supported")
        elif data['Cnf'] == '02':
            print("Authentication using PKI Certificate Exchange Supported ")
            from_client = {}
            from_client["Fn"] = '29'
            sf = int(input("Enter Requested subFunction \n 1 : verifyCertificateUnidirectionally \n 2: verifyCertificateBidirectionally "))
            print(sf)
            if sf == 2:
                from_client["sFn"] = "02" 
            else:
                from_client["sFn"] = "01"
            print(from_client["sFn"])
            from_client["CC"] = '00'
            from_client["Cert"] = ClientCert
            if from_client["sFn"] =='02':
                from_client["CCli"] = RandomGenerator()
            return from_client
    elif data["sFn"] == '01':
        print("Unidirectional Authetication")
        temp = CA.sign(data["CS"],clientPriKey)
        from_client = {}
        from_client["Fn"] = '29'
        from_client["sFn"] = '03'
        from_client["POO"] = temp
        return from_client
    elif data["sFn"] == '02':
        print("Bidirectional Authentication")
        temp = CA.sign(data["CS"],clientPriKey)
        from_client = {}
        from_client["Fn"] = '29'
        from_client["sFn"] = '03'
        from_client["POO"] = temp
        return from_client
            
            
# CA.verifyCertificate(ClientCert,CAPubKeyC )

In [7]:

import socket
import pickle
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(("127.0.0.1", 9090))
reqFrame = {"Fn":'29','sFn': '08'}
from_client = {}
data = pickle.dumps(reqFrame)
client.send(data)
data = client.recv(4096)
ChaCli=''
from_client = pickle.loads(data)
if from_client["Cnf"] == '02' and from_client["Fn"] == '69':
    tempData = Req(from_client)
    from_client = tempData
    if from_client['sFn'] == '02':
        ChaCli = from_client['CCli']
    data = pickle.dumps(from_client)
    client.send(data)
    data = client.recv(4096)
    from_client = pickle.loads(data)
if from_client["Cnf"] == '11' and from_client["Fn"] == '69':
    if from_client['sFn'] == '02':
        RecCert = from_client['ECer']
        if CA.verifyCertificate(RecCert,CAPubKeyC ):
            print("Server Certificate Verified")
            if CA.verifySign(ChaCli,from_client["POOC"],RecCert.subjectPublicKey):
                print("Certificate Verified with proof of Ownership")
#     temp = CA.sign(from_client["CS"],clientPriKey)
    tempData = Req(from_client)
    from_client = tempData
    data = pickle.dumps(from_client)
    client.send(data)
    data = client.recv(4096)
    from_client = pickle.loads(data)
if from_client['Cnf'] == "12" and from_client["Fn"] == '69':
    print("Authentication Completed")
client.close()

Authentication using PKI Certificate Exchange Supported 
Enter Requested subFunction 
 1 : verifyCertificateUnidirectionally 
 2: verifyCertificateBidirectionally 2
2
02
--------------Verification succeed

Server Certificate Verified
--------------Verification succeed

Certificate Verified with proof of Ownership
Bidirectional Authentication
Authentication Completed


In [5]:
# #hardcoded Request Response
# import socket
# import pickle
# client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# client.connect(("127.0.0.1", 9090))
# reqFrame = {"Fn":'29','sFn': '08'}
# from_client = {}
# data = pickle.dumps(reqFrame)
# client.send(data)
# data = client.recv(4096)
# from_client = pickle.loads(data)
# if from_client["Cnf"] == '02' and from_client["Fn"] == '69':
#     from_client = {}
#     from_client["Fn"] = '29'
#     from_client["sFn"] = '01'
#     from_client["CC"] = '00'
#     from_client["Cert"] = ClientCert
#     data = pickle.dumps(from_client)
#     client.send(data)
#     data = client.recv(4096)
#     from_client = pickle.loads(data)
# if from_client["Cnf"] == '11' and from_client["Fn"] == '69':
#     temp = CA.sign(from_client["CS"],clientPriKey)
#     from_client = {}
#     from_client["Fn"] = '29'
#     from_client["sFn"] = '03'
#     from_client["POO"] = temp
#     data = pickle.dumps(from_client)
#     client.send(data)
#     data = client.recv(4096)
#     from_client = pickle.loads(data)
# if from_client['Cnf'] == "12" and from_client["Fn"] == '69':
#     print("Authentication Completed")
# client.close()
#     #         data = conn.recv(4096)
# #     from_client = pickle.loads(data)
# # if from_client["sFn"] == '01':
# # #                 if from_client['CC'] == '01':
# # #                     genSessionKey =1
# #     RecCer = from_client['cert']
# #     if CA.verifyCertificate(RecCert,CAPubKeyC ):
# #         print("Certificate Verified")


Authentication Completed


In [ ]:
#  # hardcoded Request to reqfn with bidirectional authentication
# import socket
# import pickle
# client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# client.connect(("127.0.0.1", 9090))
# reqFrame = {"Fn":'29','sFn': '08'}
# from_client = {}
# data = pickle.dumps(reqFrame)
# client.send(data)
# data = client.recv(4096)
# ChaCli=''
# from_client = pickle.loads(data)
# if from_client["Cnf"] == '02' and from_client["Fn"] == '69':
#     tempData = Req(from_client)
#     from_client = tempData
# #     from_client["Fn"] = '29'
# #     from_client["sFn"] = '01'
# #     from_client["CC"] = '00'
# #     from_client["Cert"] = ClientCert
#     if from_client['sFn'] == '02':
#         ChaCli = from_client['CCli']
#     data = pickle.dumps(from_client)
#     client.send(data)
#     data = client.recv(4096)
#     from_client = pickle.loads(data)
# if from_client["Cnf"] == '11' and from_client["Fn"] == '69':
#     if from_client['sFn'] == '02':
#         RecCert = from_client['ECer']
#         if CA.verifyCertificate(RecCert,CAPubKeyC ):
#             print("Server Certificate Verified")
#             if CA.verifySign(ChaCli,from_client["POOC"],RecCert.subjectPublicKey):
#                 print("Certificate Verified with proof of Ownership")
# #     temp = CA.sign(from_client["CS"],clientPriKey)
#     tempData = Req(from_client)
#     from_client = tempData
    
# #     from_client["Fn"] = '29'
# #     from_client["sFn"] = '03'
# #     from_client["POO"] = temp
#     data = pickle.dumps(from_client)
#     client.send(data)
#     data = client.recv(4096)
#     from_client = pickle.loads(data)
# if from_client['Cnf'] == "12" and from_client["Fn"] == '69':
#     print("Authentication Completed")
# client.close()